## Created by <a href="https://github.com/yunsuxiaozi">yunsuxiaozi</a> 2025/07/23

## Competition:<a href="https://www.kaggle.com/competitions/equity-post-HCT-survival-predictions/overview">CIBMTR - Equity in post-HCT Survival Predictions</a>

## 原始代码 <a href="https://www.kaggle.com/code/yura52/cibmtr-tabm-multiloss/">[cibmtr] TabM multiloss</a>

这里为了方便读者学习最主要的东西,对代码进行了删改,删除了一些用不到的代码,同时也删除了一些原作者用来保证程序健壮性的代码,比如一些assert的语句,如果感兴趣可以阅读原始的代码,这个代码优化之后私榜能排到第20名(分数为PB:0.69465,LB:0.69232).

## 比赛用到的外部数据:

1.<a href="https://www.kaggle.com/datasets/yura52/cibmtr-folds">cibmtr-folds</a>,这里其实就是设置了数据集每折交叉验证的valid_index.

2.<a href="https://www.kaggle.com/code/yura52/cibmtr-packages">cibmtr-packages</a> 用pip download命令准备好了要用到的包,因为Kaggle比赛的环境不能联网,所以需要download然后install.

## 这里默认学习这个top方案的人都是有一定基础渴望提高的人,所以这里不会讲特别基础的东西。读者可以自行补充基础知识。

#### 更多数据挖掘比赛的top方案可以关注<a href="https://github.com/yunsuxiaozi/AI-and-competition">这里</a>。

## 方案亮点分析(我学到了什么?)

1.学会了如何使用Tabm模型进行多任务学习(Custommodel).

2.独特的多目标的损失函数(loss function).

3.对于排序标签的<a href=" http://xhslink.com/m/AlOBjtFUAJj">分位数变换(Quantile Transformer)</a>.

## 关于tabm模型的代码注释可以参考:

1. <a href="https://www.kaggle.com/code/yunsuxiaozi/hbd2025-tabm/notebook">hdb2025 tabm</a>

In [1] 安装依赖的库,这里我没有细看.

In [1]:
%pip install /kaggle/input/cibmtr-packages/autograd-1.7.0-py3-none-any.whl --quiet
%pip install /kaggle/input/cibmtr-packages/autograd-gamma-0.5.0.tar.gz --quiet
%pip install /kaggle/input/cibmtr-packages/interface_meta-1.3.0-py3-none-any.whl --quiet  # noqa: E501
%pip install /kaggle/input/cibmtr-packages/formulaic-1.1.1-py3-none-any.whl --quiet
%pip install /kaggle/input/cibmtr-packages/lifelines-0.30.0-py3-none-any.whl --quiet
%pip install /kaggle/input/cibmtr-packages/rtdl_num_embeddings-0.0.11-py3-none-any.whl --quiet  # noqa: E501
%pip install /kaggle/input/cibmtr-packages/delu-0.0.26-py3-none-any.whl --quiet

Note: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In[2] 导入一堆依赖的库,这里我没有细看.

In [2]:
import itertools
import json
import statistics
import sys
from copy import deepcopy
from pathlib import Path

import delu#对pytorch的一个扩展库.
import numpy as np
import pandas as pd
import rtdl_num_embeddings
import sklearn.preprocessing
import torch
import torch.nn as nn
from torch import Tensor
from tqdm import tqdm 

sys.path.append('/kaggle/input/cibmtr-packages')

#可以参考注释版代码:https://www.kaggle.com/code/yunsuxiaozi/hbd2025-tabm
from tabm_reference import (  
    LinearEfficientEnsemble,
    Model,
    NLinear,
    make_parameter_groups,#对模型中的参数进行分为3组,自定义分组,需要权重衰减的参数,不需要权重衰减的参数.
)


In [3] 这里就是把比赛方提供的评估指标代码搬运过来.<a href="https://www.kaggle.com/code/metric/eefs-concordance-index">eefs-concordance-index</a>

In [3]:
import pandas.api.types
from lifelines.utils import concordance_index
class ParticipantVisibleError(Exception):
    pass

def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    
    del solution[row_id_column_name]
    del submission[row_id_column_name]
    
    event_label = 'efs'
    interval_label = 'efs_time'
    prediction_label = 'prediction'
    for col in submission.columns:
        if not pandas.api.types.is_numeric_dtype(submission[col]):
            raise ParticipantVisibleError(f'Submission column {col} must be a number')
    # Merging solution and submission dfs on ID
    merged_df = pd.concat([solution, submission], axis=1)
    merged_df.reset_index(inplace=True)
    merged_df_race_dict = dict(merged_df.groupby(['race_group']).groups)
    metric_list = []
    for race in merged_df_race_dict.keys():
        # Retrieving values from y_test based on index
        indices = sorted(merged_df_race_dict[race])
        merged_df_race = merged_df.iloc[indices]
        # Calculate the concordance index
        c_index_race = concordance_index(
                        merged_df_race[interval_label],
                        -merged_df_race[prediction_label],
                        merged_df_race[event_label])
        metric_list.append(c_index_race)
    return float(np.mean(metric_list)-np.sqrt(np.var(metric_list)))

In [4] 和评估指标有关的4列特征.

In [4]:
ID_COLUMN,EVENT_COLUMN,TIME_COLUMN,GROUP_COLUMN = 'ID','efs','efs_time','race_group'

In [5]  一些基础的比较零散的函数.

In [5]:
#计算评估指标的分数.
def compute_score(df: pd.DataFrame, y_pred_risk: np.ndarray) -> float:
    #真实值是一个id,一个group和2个target
    df_true = df[[ID_COLUMN, EVENT_COLUMN, TIME_COLUMN, GROUP_COLUMN]].copy()
    #预测值是ID和prediction
    df_pred = df_true[[ID_COLUMN]].copy()
    df_pred['prediction'] = y_pred_risk
    #传入官方的评分函数得到分数
    return score(df_pred, df_true, ID_COLUMN)

#labelencoder的函数
def ordinal_encoding(series: pd.Series) -> np.ndarray:
    #查看这个数据有没有缺失值.
    has_missing_values = series.isna().any()
    #labelencoder,['a',np.nan,'b']变成[0,-1,1]
    values = series.factorize()[0].astype(np.int64)
    #如果有缺失值就加一,变成[1,0,2]
    values = values +int(has_missing_values)
    return values

#把每个part的评估指标的分数汇总乘一个字符串.
def format_metrics(
    metrics: dict, *, precision: int = 3
) -> str:
    message = []
    for part in metrics:
        message.append(f'[{part}] {round(metrics[part]["score"], precision)}')
    return ' '.join(message)

In [6] 这里是模型部分,关于模型架构的解读可以看:<a href="http://xhslink.com/m/CPLO3hXsjI">线性回归的正确打开方式</a>.

In [6]:
class CustomModel(Model):

    def __init__(self, **kwargs) -> None:
        super().__init__(**kwargs)
        assert self.arch_type == 'tabm'
        #k是输出的个数,一方面应该是整数,一方面2个任务平分输出,所以k应该是2的倍数
        assert self.k is not None
        assert self.k % 2 == 0

        del self.output

        #这里对模型进行了优化,将激活函数从relu变成了gelu,有了一点提升.
        def replace_relu_with_gelu(module):
            for name, child in module.named_children():
                if isinstance(child, nn.ReLU):
                    setattr(module, name, nn.GELU())
                else:
                    replace_relu_with_gelu(child)   # 递归
        replace_relu_with_gelu(self.backbone)

        first_block = self.backbone.blocks[0]  
        d_block = first_block[0].out_features  
        activation_layer = deepcopy(first_block[1])
        dropout_layer = deepcopy(first_block[2])
 
        self.output_event = NLinear(self.k // 2, d_block, 1)

        self.output_time = nn.Sequential(
            LinearEfficientEnsemble(
                d_block + 1,
                d_block,
                k=self.k // 2,
                ensemble_scaling_in=True,
                ensemble_scaling_out=True,
                ensemble_bias=True,
                scaling_init='ones',
            ),
            activation_layer,
            dropout_layer,
            NLinear(self.k // 2, d_block, 1),
        )

    def forward(self, x_num: Tensor, x_cat: Tensor, y_event: Tensor) -> Tensor:

        x = []
        if x_num is not None:
            x.append(x_num if self.num_module is None else self.num_module(x_num))
        if x_cat is not None:
            x.append(self.cat_module(x_cat).float())
        x = torch.column_stack([x_.flatten(1, -1) for x_ in x])
        x = x[:, None].expand(-1, self.k, -1)
        x = self.backbone(x)

        x_event, x_time = x.chunk(2, dim=1)
        x_event = self.output_event(x_event)
        if not self.training:#在模型的验证/测试部分y_event用1来代替.
            y_event = torch.ones_like(y_event)
        x_time = torch.cat(
            [x_time, y_event[:, None, None].expand(-1, self.k // 2, -1)],
            dim=-1,
        )
        x_time = self.output_time(x_time)
        
        return torch.cat([x_event, x_time], dim=1)

In [7] 这是进行多目标学习的损失函数,tabm模型对于一个数据会有k个输出,它将k/2个输出用于第一个任务,另外k/2个输出用于第二个任务。

In [7]:
def tabm_multiloss(
    #这里y_pred是有k个输出,平均分给2个任务,真实的efs和efs_time是一个值.
    y_pred: Tensor, y_event: Tensor, y_time: Tensor) -> Tensor:
    n_tasks = 2#这里有2个任务,一个是分类任务efs,一个是回归任务efs_time
    k = y_pred.shape[1]#tabm的k个输出
    k_per_task = k // n_tasks#平均每个任务有几个值.
    #将输出的预测平分给2个任务.
    y_pred_logits, y_pred_time = y_pred.chunk(n_tasks, dim=1)

    #这里分别计算了分类和回归的损失.
    cls_loss = nn.functional.binary_cross_entropy_with_logits(
        y_pred_logits.flatten(), y_event.float().repeat_interleave(k_per_task)
    )
    reg_loss = nn.functional.mse_loss(
        y_pred_time.flatten(), y_time.repeat_interleave(k_per_task)
    )

    #为了保证分类和回归同等的优化,防止出现0.01+100的情况,所以将分类损失的数值变的和回归一样.
    cls_loss = cls_loss * (reg_loss.detach().abs() / cls_loss.detach().abs())
    #平均分类和回归的损失.
    return (cls_loss + reg_loss) / 2.0

In [8] 这里是主函数,包括数据的读取、数据预处理到模型的定义和训练整个流程。注释也写的很详细了。

In [8]:
def main(config,output) :

    print("创建好输出的文件夹.")
    #先用Path()解析成对象,然后resolve变成绝对路径.
    output = Path(output).resolve()
    #在这个路径创建一个文件夹.
    output.mkdir()

    report = {
        'function': 'bin.kaggle_public_notebook_v2.main',
        'config': deepcopy(config),
    }

    #清理GPU显存.
    delu.cuda.free_memory()
    #固定随机种子,保证模型可以复现.
    delu.random.seed(config['seed'])
    #选定计算设备是GPU还是CPU.
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    print("读取训练和测试数据")
    raw_df_train = pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/train.csv')
    raw_df_test = pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/test.csv')
    print(f'{raw_df_train.shape=}\n{raw_df_test.shape=}\n')
    print(raw_df_train['efs'].mean())

    print("拼接训练数据和测试数据.")
    #训练数据和测试数据有多少行.
    raw_train_size,raw_test_size = len(raw_df_train),len(raw_df_test)
    #测试数据的efs和efs_time先随便设个值,efs设为类别1,efs_time就是从1到raw_test_size的值.
    raw_df_test['efs'] = [1.0] * raw_test_size
    raw_df_test['efs_time'] = np.arange(1, raw_test_size + 1).astype(np.float64)
    #将数据汇总在一起.
    raw_df = pd.concat([raw_df_train, raw_df_test])

    print("数据预处理.")
    df = raw_df.copy()
    for column, series in df.items():
        #数值列增加is_missing特征
        if pandas.api.types.is_numeric_dtype(series.dtype):
            is_missing = series.isna()
            if is_missing.any():#如果这列存在哪怕一个缺失值,增加新的特征,这列是不是缺失值.
                df[f'{column}:is_missing'] = is_missing.astype(np.int64)
            df[column] = series.astype(np.float64)
        else:#字符串列labelencoder.
            df[column] = ordinal_encoding(series)
    #类别型变量转int
    df['efs'] = df['efs'].astype(np.int64)
    #还原回df_train和df_test
    df_train = df.iloc[:raw_train_size].copy()
    df_test = df.iloc[raw_train_size:].copy()
    print(f'{df_train.shape=}\n{df_test.shape=}\n')

    #这个json.loads(Path(config['folds']).read_text())的shape是(10,10,2880),可能是10个种子10折每折的index.
    #这里的folds获取的就是(10,2880)每个fold的valid_index.
    folds = (
        json.loads(Path(config['folds']).read_text())
    )[config['seed']]
    n_splits = len(folds)#10折交叉验证.

    print("获取每折的训练集和验证集的index.")
    splits = []
    for val_fold_index in range(n_splits):

        #除了验证数据那折,剩下都是训练数据的index.
        train_idx = []
        for i, fold in enumerate(folds):
            if i != val_fold_index:
                train_idx.extend(fold)
        #验证数据就是其中一折
        val_idx = folds[val_fold_index]
        splits.append((train_idx, val_idx))
    
    split_predictions = []
    for split_id, (train_idx, val_idx) in enumerate(splits):
        print(f'fold {split_id}')
        #固定随机种子,保证模型可以复现.
        delu.random.seed(config['seed'] + split_id)
        split_report = {}

        print("划分训练/验证/测试集 ------------------->")
        dfs = {
            'train': df_train.iloc[train_idx].copy(),
            'val': df_train.iloc[val_idx].copy(),
            'test': df_test.copy(),
        }
        parts = list(dfs)#parts就是keys() ['train','val','test']

        print("确定特征(类别型和数值型变量) ------------------->")
        num_columns = []
        cat_columns = []
        for column, series in dfs['train'].items():
            #如果是row_id这个无用特征,或者2个target,不进行处理.
            if column in [ID_COLUMN, EVENT_COLUMN, TIME_COLUMN]:
                continue
            elif dfs['train'][column].nunique() == 1:#nunique=1也是无用特征.
                continue
            elif pandas.api.types.is_float_dtype(series.dtype):
                num_columns.append(column)
            else:
                cat_columns.append(column)

        print("数值/类别/TARGET/group的train/val/test的数据 ------------------->")
        data_numpy = {
            'x_num': {part: dfs[part][num_columns].values.copy() for part in parts},
            'x_cat': {part: dfs[part][cat_columns].values.copy() for part in parts},
            'y_event': {part: dfs[part][EVENT_COLUMN].values.copy() for part in parts},
            'y_time': {part: dfs[part][TIME_COLUMN].values.copy() for part in parts},
            'groups': {part: dfs[part][GROUP_COLUMN].values.copy() for part in parts},
        }

        print("数值列特征标准化并填充缺失值 ------------------->")
        scaler = sklearn.preprocessing.StandardScaler()
        scaler.fit(data_numpy['x_num']['train'])
        for part in parts:
            data_numpy['x_num'][part] = scaler.transform(data_numpy['x_num'][part])  
        for part in parts:
            data_numpy['x_num'][part] = np.nan_to_num(data_numpy['x_num'][part])

        print("efs_time的分位数变换 ------------------->")
        #训练集的efs_time的值,训练集efs=1作为mask.
        y_time_train = data_numpy['y_time']['train'].copy()
        print(pd.DataFrame({'efs':y_time_train})['efs'].describe())
        event_mask = data_numpy['y_event']['train'].astype(bool)
        #将efs_time按照efs=0和efs=1的数据单独转成正态分布.
        #参考资料:https://zhuanlan.zhihu.com/p/1911421888795776012
        for mask in [event_mask, ~event_mask]:
            y_time_train[mask] = (
                sklearn.preprocessing.QuantileTransformer(
                    n_quantiles=len(y_time_train)// 30,
                    output_distribution='normal',
                    subsample=1000000,
                )
                .fit(y_time_train[mask][:, None])
                .transform(y_time_train[mask][:, None])
                .squeeze(-1) 
            )
        
        #训练数据保存转换后的efs_time.
        data_numpy['y_time']['train'] = y_time_train
        print(pd.DataFrame({'efs':y_time_train})['efs'].describe())

        #key就是num,cat,group,target之类的,part是train/val/test,value就是数据.
        print("数据的tensor字典 ------------------->")
        data = {
            key:{
                part: torch.as_tensor(
                    value,
                    dtype={'float64': torch.float32, 'int64': torch.int64}[
                        str(value.dtype)
                    ],
                    device=device,
                )
                for part, value in data_numpy[key].items()
            }
            for key in data_numpy.keys()
        }
        #每部分数据train/val/test有多少数据量.
        size = {part: len(data['y_time'][part]) for part in parts}

        print("加载模型. ------------------->")
        model = CustomModel(
            n_num_features=data['x_num']['train'].shape[1],#数值型变量每个变量1个维度
            #类别型变量维度看onehot.
            cat_cardinalities=[len(torch.unique(x)) for x in data['x_cat']['train'].T],
            n_classes=None,#不是分类任务.
            **config['model'],#模型的其他参数.
            bins=None,#如果不对连续型变量按区间分桶,根据传入的参数去rtdl_num_embeddings里找对应的模块
        ).to(device)#移动到GPU设备.
        print(model)

        #这是y_pred=model(X)的函数.
        def apply_model(part: str, idx: None | Tensor = None) -> Tensor:
            #获取part(train/val/test)的数值变量、类别变量,target
            x_num,x_cat,y_event = data['x_num'][part],data['x_cat'][part],data['y_event'][part]
            #获取一个batch_size的数据
            if idx is not None:
                x_num,x_cat,y_event = x_num[idx],x_cat[idx],y_event[idx]
            #返回模型输出的结果.
            return model(x_num, x_cat, y_event).squeeze(-1)

        print("模型训练相关的参数. ------------------->")
        #先将模型的参数分组,然后进行优化. 学习率和权重衰减.
        optimizer = torch.optim.AdamW(
            params=make_parameter_groups(model), **config['optimizer']
        )
        #梯度L2范数缩放到以内.
        gradient_clipping_norm = config['gradient_clipping_norm']

        #训练集和验证集的一批次数据,验证数据可以大一点
        batch_size,eval_batch_size = config['batch_size'],config['eval_batch_size']
        #训练集分为多少次完成一个epoch的训练.
        epoch_size = np.ceil(size['train'] / batch_size)
        #用来打乱训练集输入顺序的工具,设好训练的设备和随机种子,保证可以复现.
        batch_generator = torch.Generator(device).manual_seed(config['seed'] + split_id)
        
        #早停机制,模型在验证集上的效果n次没有提升就停止训练,指标越大越好.
        early_stopping = delu.tools.EarlyStopping(
            patience=config['patience'], mode='max'
        )
        best_checkpoint = None

        #装饰器,这时推理阶段,不需要计算梯度,不需要反向传播,可以减少内存消耗,提高计算速度.
        @torch.inference_mode()
        #得到part的指标分数和预测结果.
        def evaluate(parts):#parts=['train','val','test']
            model.eval()#模型转成评估模式.

            metrics = {}
            predictions = {}
            for part in parts:
                #模型对全部(训练/验证/测试)数据分批次进行推理,推理之后再拼接.
                submodel_y_pred = (
                    torch.cat(
                        [   apply_model(part, idx)
                        for idx in torch.arange(size[part], device=device).split(eval_batch_size)
                        ]
                    )
                )
                #推理之后将预测结果分成2部分.
                submodel_y_pred_logits, submodel_y_pred_time = submodel_y_pred.chunk(2, dim=1)
                #得到efs和efs_time的预测结果.
                y_pred_probs = submodel_y_pred_logits.sigmoid().mean(1)
                y_pred_time = submodel_y_pred_time.mean(1)
                #CIBMTR这场比赛特有的组合公式,得到最终的排序结果.
                y_pred_risk = (y_pred_probs * (-y_pred_time).sigmoid()).cpu().numpy()

                #测试集不能计算分数,否则计算分数.
                if part != 'test':
                    metrics[part] = {'score': compute_score(dfs[part], y_pred_risk)}
                #保存预测结果.
                predictions[part] = y_pred_risk
            return metrics, predictions

        print("模型的训练. ------------------->")
        for i in range(config['n_epochs']):
            print(f"epoch{i+1}")
            model.train()#模型调成训练模式.
            #取出一批次的数据
            for batch_idx in tqdm(torch.randperm(size['train'], device=device).split(batch_size),):
                optimizer.zero_grad()#梯度清空.
                #计算这个批次的损失.
                loss = tabm_multiloss(
                    y_pred=apply_model('train', batch_idx),
                    y_event=data['y_event']['train'][batch_idx],
                    y_time=data['y_time']['train'][batch_idx],
                )
                loss.backward()#损失反向传播
                #梯度进行缩放,防止梯度爆炸
                nn.utils.clip_grad.clip_grad_norm_(
                    model.parameters(), gradient_clipping_norm
                )#进行更新参数.
                optimizer.step()
            #计算验证集上的评估指标.
            metrics, _ = evaluate(['val'])
            #要么是第一个epoch没有赋值上去,要么比之前的score好,才会更新评估指标和模型的参数.
            if (  'metrics' not in split_report
                or metrics['val']['score'] > split_report['metrics']['val']['score']
            ):
                split_report['metrics'] = metrics
                best_checkpoint = deepcopy({'model': model.state_dict()})
            #早停是每个epoch都要更新,因为要看n次有没有提升.
            early_stopping.update(metrics['val']['score'])
            #如果在验证集上的效果n次没有提升,就停止.
            if early_stopping.should_stop():
                break

        #训练完成之后加载最好的模型.
        model.load_state_dict(best_checkpoint['model'])
        #得到训练/验证集上的评估指标和训练/验证/测试集上的预测结果.
        metrics, predictions = evaluate(['train', 'val', 'test'])
        #更新训练/验证集的评估指标,训练/验证/测试集的预测.
        split_report['metrics'] = metrics
        split_predictions.append(predictions)
        #汇总split_report
        report.setdefault('split_reports', []).append(split_report)

    print("最终的预测结果.")
    #用于统计训练/验证/测试集的prediction
    predictions = {
        'val': np.zeros(raw_train_size, dtype=np.float32),
        'test': np.zeros(raw_test_size, dtype=np.float32),
    }
    #训练集训练了n-1次,验证集验证了1次,测试集推理了n次,所以汇总数据的时候需要乘对应的比例.
    for (train_idx, val_idx), this_split_predictions in zip(splits, split_predictions):
        predictions['val'][val_idx] += this_split_predictions['val']
        predictions['test'] += this_split_predictions['test']/n_splits

    print("构造提交文件.")
    df = pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/sample_submission.csv')
    df['prediction'] = predictions['test']
    df.to_csv('submission.csv', index=False)
    #输出每折交叉验证的score.
    for split_id, split_report in zip(range(n_splits), report['split_reports']):
        print(
            f'Split {split_id}: ' + format_metrics(split_report['metrics'], precision=4)
        )

In [9] 程序最终的运行代码,前面的参数用于保证模型可以复现,后面设置一些基础的参数之后就调用main函数。

In [9]:
#CPU只使用1个线程,避免多线程带来的随机调度差异
torch.set_num_threads(1)
#关闭tf32加速(tf32加速通过将FP32降到FP16,降低精度加快计算速度但是带来更大的误差)
torch.backends.cuda.matmul.allow_tf32 = False
#针对cuDNN卷积操作,也不使用加速但是会增加误差的操作.
torch.backends.cudnn.allow_tf32 = False 
#关闭cuDNN 的自动算法搜索(benchmark),如果每次运行程序的时候都从多个算法中挑速度最快的算法,可能会选到不同算法,程序无法复现.
torch.backends.cudnn.benchmark = False 
#强制cuDNN里所有的卷积都使用确定性算法,保证结果可以复现.
torch.backends.cudnn.deterministic = True  

config= {
    'seed': 0,
    'folds': '/kaggle/input/cibmtr-folds/stratified-group-10.json',
    'gradient_clipping_norm':0.9,#梯度缩放因子,比如梯度的L2范数为2,乘1/2缩放为1,防止梯度爆炸.
    'batch_size': 1024,
    'eval_batch_size': 8192,
    'n_epochs': 1024,#1024是遇到早停之后再停下。
    'patience': 20,
    'model': {
        'arch_type': 'tabm',
        'k': 48,
        'backbone': {
            'type': 'MLP',
            'n_blocks': 2,
            'd_block': 1024,
            'dropout': 0.024,
        },
        'num_embeddings': {
            'type': 'PeriodicEmbeddings',#使用周期性嵌入
            'n_frequencies':27,#生成多少个正余弦分量.
            'd_embedding': 42,#embedding的维度.
            'frequency_init_scale':0.25,#初始化频率参数的标准差.
            'lite': False,#是否使用轻量版,据我理解应该是数学上的欧拉公式.
        },
    },
    'optimizer': {
        'lr': 0.0015,
        'weight_decay': 0.02,
    },
}
main(config, f'output-0')

创建好输出的文件夹.
读取训练和测试数据
raw_df_train.shape=(28800, 60)
raw_df_test.shape=(3, 58)

0.5393055555555556
拼接训练数据和测试数据.
数据预处理.
df_train.shape=(28800, 80)
df_test.shape=(3, 80)

获取每折的训练集和验证集的index.
fold 0
划分训练/验证/测试集 ------------------->
确定特征(类别型和数值型变量) ------------------->
数值/类别/TARGET/group的train/val/test的数据 ------------------->
数值列特征标准化并填充缺失值 ------------------->
efs_time的分位数变换 ------------------->
count    25920.000000
mean        23.199379
std         24.745115
min          0.333000
25%          5.608000
50%          9.767500
75%         35.070000
max        156.819000
Name: efs, dtype: float64
count    25920.000000
mean        -0.000058
std          1.000690
min         -5.199338
25%         -0.674402
50%         -0.000290
75%          0.674523
max          5.199338
Name: efs, dtype: float64
数据的tensor字典 ------------------->
加载模型. ------------------->
CustomModel(
  (num_module): PeriodicEmbeddings(
    (periodic): _Periodic()
    (linear): _NLinear()
    (activation): ReLU()
  )
  (cat_mod

100%|██████████| 26/26 [00:03<00:00,  7.24it/s]


epoch2


100%|██████████| 26/26 [00:02<00:00,  8.77it/s]


epoch3


...

epoch55


100%|██████████| 26/26 [00:02<00:00,  8.77it/s]


epoch56


100%|██████████| 26/26 [00:02<00:00,  8.76it/s]


最终的预测结果.
构造提交文件.
Split 0: [train] 0.7118 [val] 0.6776
Split 1: [train] 0.786 [val] 0.6797
Split 2: [train] 0.7353 [val] 0.6795
Split 3: [train] 0.7286 [val] 0.6732
Split 4: [train] 0.733 [val] 0.6809
Split 5: [train] 0.7401 [val] 0.6777
Split 6: [train] 0.7401 [val] 0.6863
Split 7: [train] 0.7346 [val] 0.6807
Split 8: [train] 0.7861 [val] 0.6835
Split 9: [train] 0.765 [val] 0.6755
